In [838]:
#3/13/18

import pandas as pd
import tensorflow as tf
import numpy as np
import math
from sklearn.metrics import roc_auc_score

In [848]:
def OH_F(ITID, items_l):
        oh_v = np.zeros(len(items_l))
        iidx = 0
        for item in items_l:
            if ITID == item:
                oh_v[iidx] = 1
            iidx += 1
        return (pd.DataFrame(oh_v))
    
def myGenerator_tr():
    ##Yields tuple for rows of chartevents csv file for a single HADM_ID each time it is called. Tuple contains HADM_ID and 3 Nx1 np.arrays for item id chartime and valuenum respectively  
    #(Assumes csv is sort by HADM_ID)
    a_df = pd.read_csv('/soe/dcjenkin/mimic3/mimic3/ADMISSIONS.csv')
    chart_path = '/soe/dcjenkin/mimic3/mimic3/CHARTEVENTS_tr.csv'
    item_list = [220045,211,51,8368,807,227037,223761,227059,220277,220210,198,223835,1817]
    ADM_DTH_Dict = {}
    
    for i, row in a_df.iterrows():
        ADM_DTH_Dict[row['HADM_ID']] = [row['ADMITTIME'], row['DISCHTIME'], row['DEATHTIME']]
    first_bool = True
    
    for df in pd.read_csv(chart_path ,sep=',', chunksize=1, usecols=['HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM']):
        #Loop through csv line by line appending to HADM_ID_chunk.
        if df.iloc[0,1] in item_list: #IF ITID is in item_IDs list
            if not pd.isnull(df.iloc[0,3]): #If the valuenum is not null
                OH_V = OH_F(df.iloc[0,1], item_list)
                df.reset_index(drop=True, inplace=True)
                df = pd.concat([df, OH_V.T], axis = 1)
                if first_bool:
                    last_row_HADM_ID = df.iloc[0,0] #HADM_ID
                    first_bool = False
                    HADM_ID_chunk = df
                else:
                    next_row_HADM_ID = df.iloc[0,0]
                    #When HADM_ID changes, yield HADM_ID_chunk as np array then start new chunk.
                    if next_row_HADM_ID == last_row_HADM_ID:
                        HADM_ID_chunk = HADM_ID_chunk.append(df)
                    else:
                        chunk_array = np.array(HADM_ID_chunk)
                        ADM_DTH_array = np.array(ADM_DTH_Dict[last_row_HADM_ID])
                        
                        if pd.isnull(ADM_DTH_array[2]):
                            H_Mortality = 0
                        else: 
                            H_Mortality = int(pd.Timestamp(ADM_DTH_array[2]) <= pd.Timestamp(ADM_DTH_array[1]))
                            
                        yield (chunk_array[0,0], chunk_array[:,1] ,chunk_array[:,2], chunk_array[:,3:], ADM_DTH_array, H_Mortality)
                        HADM_ID_chunk = df
                    last_row_HADM_ID = next_row_HADM_ID
                    

def myGenerator_test():
    ##Yields tuple for rows of chartevents csv file for a single HADM_ID each time it is called. Tuple contains HADM_ID and 3 Nx1 np.arrays for item id chartime and valuenum respectively  
    #(Assumes csv is sort by HADM_ID)
    a_df = pd.read_csv('/soe/dcjenkin/mimic3/mimic3/ADMISSIONS.csv')
    chart_path = '/soe/dcjenkin/mimic3/mimic3/CHARTEVENTS_test.csv'
    item_list = [220045,211,51,8368,807,227037,223761,227059,220277,220210,198,223835,1817]
    ADM_DTH_Dict = {}
    
    for i, row in a_df.iterrows():
        ADM_DTH_Dict[row['HADM_ID']] = [row['ADMITTIME'], row['DISCHTIME'], row['DEATHTIME']]
    first_bool = True
    
    for df in pd.read_csv(chart_path ,sep=',', chunksize=1, usecols=['HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM']):
        #Loop through csv line by line appending to HADM_ID_chunk.
        if df.iloc[0,1] in item_list: #IF ITID is in item_IDs list
            if not pd.isnull(df.iloc[0,3]): #If the valuenum is not null
                OH_V = OH_F(df.iloc[0,1], item_list)
                df.reset_index(drop=True, inplace=True)
                df = pd.concat([df, OH_V.T], axis = 1)
                if first_bool:
                    last_row_HADM_ID = df.iloc[0,0] #HADM_ID
                    first_bool = False
                    HADM_ID_chunk = df
                else:
                    next_row_HADM_ID = df.iloc[0,0]
                    #When HADM_ID changes, yield HADM_ID_chunk as np array then start new chunk.
                    if next_row_HADM_ID == last_row_HADM_ID:
                        HADM_ID_chunk = HADM_ID_chunk.append(df)
                    else:
                        chunk_array = np.array(HADM_ID_chunk)
                        ADM_DTH_array = np.array(ADM_DTH_Dict[last_row_HADM_ID])
                        
                        if pd.isnull(ADM_DTH_array[2]):
                            H_Mortality = 0
                        else: 
                            H_Mortality = int(pd.Timestamp(ADM_DTH_array[2]) <= pd.Timestamp(ADM_DTH_array[1]))
                            
                        yield (chunk_array[0,0], chunk_array[:,1] ,chunk_array[:,2], chunk_array[:,3:], ADM_DTH_array, H_Mortality)
                        HADM_ID_chunk = df
                    last_row_HADM_ID = next_row_HADM_ID
                    
def map_func1(HADM_ID, ITEM_IDs, Charttimes, Values, ADM_DSC_DTH, H_Mort):
    num_meas = tf.shape(Values)[0]
    #finds the number of groups.  Each group contains n_steps = 100 measurements per patient
    #(eg if there are 637 measurements for a patient that would form 7 groups).
    numgroup_minus_one = tf.reduce_max([tf.to_int32(tf.ceil(num_meas/100))-1, 0])
    #generates labels for each group indicating mortality at the end of that group.  Only the final group can be non-zero as the patients are still alive if measurements are still being taken.
    labellist = tf.zeros([numgroup_minus_one], tf.int32)
    labellist = tf.concat([labellist, [H_Mort]], 0)
    
    paddings = tf.constant([[0, 100,], [0, 0]])
    Values = tf.pad(Values, paddings, "CONSTANT")
    return HADM_ID, ITEM_IDs, Charttimes, Values, ADM_DSC_DTH, H_Mort, labellist, num_meas

def map_func2(HADM_ID, ITEM_IDs, Charttimes, Values, ADM_DSC_DTH, H_Mort, labellist, num_meas):
    #find the max number of measurements for patients in the batch.
    mlen = tf.reduce_max(num_meas)
    #round up to nearest 100 and take only the values up to this index.  (All values after rmlen should be zeros from the padding)
    rmlen = tf.to_int32(tf.ceil(mlen/100)*100)
    Values = Values[:,0:rmlen]
    #split up into groups of 100 measurements each
    Values = tf.reshape(Values, [2,-1,100,14])
    #transpose to match format for rnn.
    Values = tf.transpose(Values, perm=[1, 0, 2, 3]) #(number of groups, batch size, n_steps, number of feature types)
    labellist = tf.transpose(labellist, perm=[1,0])

    return HADM_ID, ITEM_IDs, Charttimes, Values, ADM_DSC_DTH, H_Mort, labellist, num_meas

In [849]:
tf.reset_default_graph()

learning_rate = 0.001

n_input = 14   # (OH encoded vector of length 13, value)

n_steps = 100  #100 timesteps per batch

n_hidden = 100  # hidden layer num of features

n_classes = 1  # Mortality in hospital.  The network will put out one value between 0 and 1 at each timestep indicating the prediction whether the patient has lived or died.  (0 for lived, 1 for died)

batch_size = 2

# X_lengths = tf.placeholder(tf.int32, [None], name='X_lengths')  #feed in the unpadded sequence length for each batch

# tf Graph input (X is a 3D array of inuts (number of patients x number of measurements x 14 for OH vector and value))
X = tf.placeholder(tf.float32, [None, n_steps, n_input], name='X')
# y is a 1D array of labels shape = (number of patients). Here for a given patient the label is the same at each time step since it just indicates whether that patient died in the hospital.
y = tf.placeholder(tf.float32, [None], name='y')
y2 = tf.reshape(y, shape = (batch_size,-1))

#Weights and Biases to map hidden state to n_classes predictions
w1 = tf.Variable(tf.random_normal([n_hidden, n_classes]), name='w1')
b1 = tf.Variable(tf.random_normal([n_classes]), name='w2')


def length(sequence):
    #returns a vector of sequence lengths for each patient within the batch segment
    used = tf.sign(tf.reduce_max(tf.abs(sequence), 2))
    length = tf.reduce_sum(used, 1)
    length = tf.cast(length, tf.int32)
    return length

X_l = length(X)

# Define a lstm cell with tensorflow
lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=False)

state = tf.Variable(lstm_cell.zero_state(batch_size, tf.float32), trainable=False)

outputs, states = tf.nn.dynamic_rnn(
        cell=lstm_cell,
        dtype=tf.float32,
        sequence_length=X_l,
        inputs=X,
        initial_state=state)

state_op = tf.assign(state, states)


def last_relevant(output, length):
    #returns outputs for each patient at the indexes given in length
    batch_size = tf.shape(output)[0]
    max_length = tf.shape(output)[1]
    out_size = int(output.get_shape()[2])
    index = tf.range(0, batch_size) * max_length + (length - 1)
    flat = tf.reshape(output, [-1, out_size])
    relevant = tf.gather(flat, index)
    return relevant


# Remove patients from batch that have no measurements for that batch (ie the length of the sequence is zero for that patient for that batch).
# mask = np.array([False, True])
mask = X_l > 0
outputs2 = tf.boolean_mask(outputs, mask)
X_l2 = tf.boolean_mask(X_l, mask)
y3 = tf.boolean_mask(y2, mask)
masksum = tf.reduce_sum(tf.cast(mask, tf.float32))

# outputs2 = outputs[1]
# X_l 

rel = last_relevant(outputs2, X_l2)

pred1 = tf.matmul(rel, w1)

pred2 = tf.add(pred1,b1)

sigmoid_pred = tf.sigmoid(pred2)

cost = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(targets = y3,logits = pred2, pos_weight= 1), name='cost')

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost, name='optimizer')

/soe/dcjenkin/workspace/venv/lib/python3.4/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [851]:
ds_train = tf.data.Dataset.from_generator(
    myGenerator_tr, (tf.int32, tf.int32, tf.string, tf.float32, tf.string, tf.int32), (tf.TensorShape([]), tf.TensorShape([None]),tf.TensorShape([None]),tf.TensorShape([None,14]), tf.TensorShape([3]), tf.TensorShape([])))

ds_train = ds_train.map(map_func1, num_parallel_calls=10)

train_dataset = ds_train.shuffle(buffer_size=10)
train_dataset = ds_train.repeat()

ds_train = ds_train.padded_batch(2, padded_shapes=([],[None],[None],[None,14],[None], [], [None], []))

ds_train = ds_train.map(map_func2, num_parallel_calls=10)

train_next_element = ds_train.make_one_shot_iterator().get_next()

ds_test = tf.data.Dataset.from_generator(
    myGenerator_test, (tf.int32, tf.int32, tf.string, tf.float32, tf.string, tf.int32), (tf.TensorShape([]), tf.TensorShape([None]),tf.TensorShape([None]),tf.TensorShape([None,14]), tf.TensorShape([3]), tf.TensorShape([])))

ds_test = ds_test.map(map_func1, num_parallel_calls=10)

ds_test = ds_test.padded_batch(2, padded_shapes=([],[None],[None],[None,14],[None], [], [None], []))

ds_test = ds_test.map(map_func2, num_parallel_calls=10)

test_iterator = ds_test.make_initializable_iterator()
test_next_element = test_iterator.get_next()

train_dataset = ds_train.prefetch(5)
test_dataset = ds_test.prefetch(5)

In [852]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    AUC = []
    for i in range(500):
        print(i)
#         print(sess.run(state))
        train_value = sess.run(train_next_element)
        #initializes hidden/cell states to zero at the beginning of each patient batch.
        sess.run(tf.variables_initializer([state]))

        for j in np.arange(np.shape(train_value[3])[0]):
            #take the jth group's input and labels to feed into the LSTM.
            batch_x, batch_y = train_value[3][j], train_value[6][j]

            feed = {X: batch_x, y: batch_y}

            #run the session.  (optimizer will update parameters).  State op will update the hidden state and cell state to the values at the end of the previous group.
            result = sess.run([optimizer, state_op, y3, sigmoid_pred], feed_dict=feed)
#             print(result[2])
#             print(result[3])
            
        if (i+1)%100 == 0:
            #save_path = saver.save(sess, "../models/model5" + str(m) + ".ckpt")
            #m +=1
            #create arrays for the prediction outputs and labels.
            preds = np.array([])
            labs = np.array([])
            #initialize the test iterator to start at the beginning
            sess.run(test_iterator.initializer)
            for i in np.arange(50):
                try:
                    test_value = sess.run(test_next_element)
    #                 print(sess.run(state))
                    sess.run(tf.variables_initializer([state]))
    #                 print(sess.run(state))
    #                 print(test_value[1])

                    for k in np.arange(np.shape(test_value[3])[0]):
                        #take the k'th group's input and labels to feed into the LSTM.
                        batch_x, batch_y = test_value[3][k], test_value[6][k]

                        feed = {X: batch_x, y: batch_y}

                        result = sess.run([sigmoid_pred, y3, state_op], feed_dict=feed)
                        #append the predictions and labels to the arrays
                        preds = np.append(preds, result[0].flatten())
                        labs = np.append(labs, result[1].flatten())
                except tf.errors.OutOfRangeError:
                    break
            #Compute and print the area under receiver operator curve for predictions and labels
            AUC.append(roc_auc_score(labs, preds))
            print('Test AUC:' + str(AUC))

0
[[0.]
 [0.]]
[[0.99980026]
 [0.99950886]]
[[0.]
 [0.]]
[[0.9996393]
 [0.9993036]]
[[1.]]
[[0.994532]]
Test AUC:[0.15]
1
[[0.]
 [0.]]
[[0.98509926]
 [0.9957248 ]]
[[0.]]
[[0.9883797]]
[[0.]]
[[0.9753067]]
[[0.]]
[[0.95138985]]
[[0.]]
[[0.938113]]
[[0.]]
[[0.8882503]]
[[0.]]
[[0.7448517]]
[[0.]]
[[0.1700356]]
[[0.]]
[[0.32377034]]
[[0.]]
[[0.3610526]]
[[0.]]
[[0.25793266]]
[[0.]]
[[0.07666402]]
2
[[0.]
 [0.]]
[[0.04267583]
 [0.01460495]]
[[0.]
 [0.]]
[[0.03796801]
 [0.0066658 ]]
[[0.]]
[[0.01393086]]
[[0.]]
[[0.00583404]]
3
[[0.]
 [1.]]
[[0.0079277 ]
 [0.02778624]]
[[0.]]
[[0.0005376]]
[[0.]]
[[0.00366715]]
4
[[0.]
 [0.]]
[[0.00054498]
 [0.00357448]]
[[0.]
 [1.]]
[[0.00233402]
 [0.00358052]]
[[0.]]
[[0.0002302]]
[[0.]]
[[0.00033845]]
[[1.]]
[[0.00286164]]
5
[[0.]
 [0.]]
[[0.00256737]
 [0.00066697]]
[[0.]
 [0.]]
[[0.00414534]
 [0.00352834]]
6
[[0.]
 [0.]]
[[0.00604642]
 [0.00264507]]
[[0.]]
[[0.00608156]]
7
[[1.]
 [0.]]
[[0.00011032]
 [0.00047556]]
[[0.]]
[[0.00622943]]
[[1.]]
[[0.00290